In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](http://)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
colors = ['#B90276','#50237F', '#005691', '#008ECF','#E20015', '#00A8B0', '#78BE20', '#006249', '#525F6B']

sns.set_palette(sns.color_palette(colors))

In [ ]:
data = pd.read_csv('/kaggle/input/concrete-compressive-strength/Concrete Compressive Strength.csv', sep=',')
df = pd.DataFrame(data)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.rename(columns={'Cement (component 1)(kg in a m^3 mixture)': 'cement_mix',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)':'Blast Furnace Slag' ,
       'Fly Ash (component 3)(kg in a m^3 mixture)': 'Fly Ash',
       'Water  (component 4)(kg in a m^3 mixture)': 'Water',
       'Superplasticizer (component 5)(kg in a m^3 mixture)': 'Superplastcizer',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': 'Coarse_Agg',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)':'Fine_Agg', 'Age (day)': 'Age_in_Days',
       'Concrete compressive strength(MPa, megapascals) ':'Strength in MPa'})

In [ ]:
i=1
plt.figure(figsize=(20,15))
for col in df.columns:
    plt.subplot(4,3,i)
    sns.histplot(data=df, x=col)
    i+=1

In [ ]:
plt.figure(figsize = (10,7))
corr = df.corr()
sns.heatmap(corr, linewidth = 0.7, cmap ='coolwarm', annot = True)

In [ ]:
df1 = df.copy()


In [ ]:
df1['strength_Range']=pd.qcut(df1['Strength in MPa'], q = 5)
#df1 = df.drop(['Strength in MPa'], axis = 1)

In [ ]:
df.columns

In [ ]:
i = 1
plt.figure(figsize=(20,15))
col = ['cement_mix', 'Blast Furnace Slag', 'Fly Ash', 'Water',
       'Superplastcizer', 'Coarse_Agg', 'Fine_Agg', 'Age_in_Days',
       'Strength in MPa']
for col in col:
    plt.subplot(4,3,i)
    sns.scatterplot(x = 'cement_mix', y= col, data = df1, hue = 'strength_Range')
    i+=1
   

***I have created a new column "Strength_Range" so that the pair plot gives more clarity***

In [ ]:
sns.pairplot(df1, hue = 'strength_Range')

In [ ]:
i=1
plt.figure(figsize = (15,20))
for col in df.columns:
    plt.subplot(4,3,i)
    sns.boxplot(x = df[col], data = df)
    i+=1

In [ ]:
df1.columns

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(121, projection = '3d')

z = df1['cement_mix']
y = df1['Superplastcizer']
x = df1['Strength in MPa']

ax.scatter(x, y, z)
ax.set_xlabel("Strength in MPa")
ax.set_ylabel("Superplastcizer")
ax.set_zlabel("cement_mix")
plt.title("Strength in MPa, Cement MIX & Plasticizer")

ax = fig.add_subplot(122, projection = '3d')

z = df1['cement_mix']
y = df1['Water']
x = df1['Strength in MPa']

ax.scatter(x, y, z)
ax.set_xlabel("Strength Range")
ax.set_ylabel("Water")
ax.set_zlabel("Cement Mix")
plt.title("Strength in MPa, Cement MIX & Water")

plt.show()


In [ ]:
plt.figure(figsize = (15,10))
sns.boxplot(x = 'Age_in_Days', y = 'Strength in MPa', data = df1, hue = 'strength_Range')

In [ ]:
X = df.drop(['Strength in MPa'], axis = 1)
y = df['Strength in MPa']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =43, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, BayesianRidge, ridge_regression 
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor


dtr = DecisionTreeRegressor()
xgbr = XGBRegressor(verbosity=0, random_state = 43)
lr = LogisticRegression(max_iter=1000)
rd = Ridge()
la = Lasso()
byrd = BayesianRidge()

models = [rd, la, byrd, dtr,xgbr]
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mod = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2_score = metrics.r2_score(y_test, y_pred).round(3)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred)).round(3)
    print('\n', model,'\n', 'R2_score:', r2_score,'\n', 'RMSE:', RMSE, '\n')
